# Annoy Analysis:
Annoy (Approximate Nearest Neighbors Oh Yeah) is an efficient technique for finding nearest neighbors in large datasets, particularly in applications involving vectors, such as machine learning and information retrieval.

## How Annoy Works:

Data (typically vectors) are added to an index. Each vector represents a numerical representation of an item.

Tree Division: Annoy builds multiple search trees, where each tree is constructed based on a random sample of the data. The idea is to partition the space into regions, allowing for faster neighbor searches.

Search Algorithm:
When querying a vector, Annoy searches through the constructed trees. It traverses the trees to find the nearest neighbors.
The algorithm is designed to be fast, using approximation, which means it might not always find the exact nearest neighbors but frequently finds very close neighbors efficiently.

In [1]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553317 sha256=73d844c321d7b78727064a7817ebe876cdbd7a06d804b5869414894b5f13a50e
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from annoy import AnnoyIndex

In [3]:
rice_df = pd.read_excel('/content/rice_w2vec_3mer_dataset.xlsx')

In [14]:
maize_df = pd.read_excel('maize_w2vec_3mer_dataset.xlsx')

In [15]:
df = rice_df._append(maize_df, ignore_index = True)

In [19]:
df_drought = df.query('stress == "-" or stress == "drought"').replace('-', 'control')

In [17]:
dimension = 64

In [20]:
df_vec = df_drought.drop(['seq','tissue','chr','start','end','strand','start_anno','circID', 'gene', 'isoform', 'width', 'detection_score', 'stress_detection_score','end_anno', 'antisense', 'algorithm', 'exonSeq',	'predAA',	'miRNA',	'superCircRNARegion'], axis=1)
df_vec

,circName,stress,wc_3mer_1,wc_3mer_2,wc_3mer_3,wc_3mer_4,wc_3mer_5,wc_3mer_6,wc_3mer_7,wc_3mer_8,...,wc_3mer_55,wc_3mer_56,wc_3mer_57,wc_3mer_58,wc_3mer_59,wc_3mer_60,wc_3mer_61,wc_3mer_62,wc_3mer_63,wc_3mer_64
0,osa-circ1-OS01T0723400,control,-8.660600,-17.171770,-20.124799,10.607480,-0.194991,1.568289,3.896590,7.622823,...,-21.976927,-12.450649,-8.376489,-9.521074,28.565338,7.251650,-18.073032,-6.670159,-3.779034,1.302005
1,osa-circ2-OS03T0223400,control,-4.623322,-42.058749,-73.491472,-51.222795,83.974883,-47.219114,-20.222901,29.187759,...,24.437080,-68.490463,20.158427,14.177653,81.437811,-4.524170,21.867150,-10.532807,-39.172970,63.339612
2,osa-circ3-OS11T0210300,control,-23.932238,-17.885510,-54.202845,15.682650,-30.424396,13.430986,-15.981544,27.324571,...,-34.355928,13.556080,-50.537661,-30.186757,64.202528,54.450606,-24.835176,2.558572,-9.287752,-25.051924
3,osa-circ4-OS02T0200900,control,-42.541243,-44.121847,-54.312436,-37.621338,53.242821,6.651359,-9.533580,22.141600,...,74.941825,-37.716935,-19.644171,36.562092,62.745595,14.111886,46.643984,-12.691884,40.033801,-0.980058
4,osa-circ5-OS05T0494800,control,-9.970732,20.265143,-4.832509,39.979788,-9.345131,-2.708796,13.024784,8.689754,...,-49.435175,33.124836,-4.937962,-21.755058,-0.672400,39.981767,-57.044162,-18.266155,-12.560292,-41.720146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101828,zma-circ38781--,control,-31.971423,46.936561,37.374460,24.961271,-20.605168,-39.592321,42.670800,-18.597109,...,55.547672,-34.047023,-53.817768,24.989076,78.595877,12.960359,41.452756,24.087117,-24.891010,24.140963
101829,zma-circ38782--,control,-7.262153,24.640844,-6.384969,-38.287346,7.022847,-3.239892,-1.993565,-7.791742,...,16.006142,-7.553426,9.093510,10.978614,37.843186,18.243088,-5.548205,-11.582676,-3.474853,13.129991
101830,zma-circ38783--,control,-75.265843,89.033060,39.742789,1.053011,-37.685334,-40.503932,34.544268,-78.418910,...,111.958343,-4.080127,-47.411417,19.781623,169.187258,13.342812,63.040171,58.404351,-10.334180,49.309830
101831,zma-circ38784--,control,82.905541,27.218369,-27.637714,-128.632740,143.095296,-70.905655,106.884221,108.640803,...,67.985388,16.778138,-142.177544,110.127476,96.632890,181.435268,-39.288972,4.967059,35.011970,79.163880


In [21]:
df_vec['stress'].value_counts()

,count
stress,
control,70996
drought,12030


In [23]:
drought_sequences = df_vec[df_vec['stress'] == 'drought']
control_sequences = df_vec[df_vec['stress'] == 'control']

# Controlled sample of "control" sequences
control_sample = control_sequences.sample(n=12030, random_state=40)

# Combine "drought" and "control" sequences
combined_sequences = pd.concat([drought_sequences, control_sample])

# Initialize the Annoy index
t = AnnoyIndex(dimension, 'angular')
for i in range(len(combined_sequences)):
    t.add_item(i, combined_sequences.iloc[i, 2:2 + dimension].values)
t.build(10)

# Initialize data structures
similarities = {}
n_neighbors = 5  # Desired number of neighbors

valid_neighbors_count = []  # Count of valid neighbors
drought_without_neighbors = 0  # Counter for sequences with no valid neighbors
total_neighbors_found = 0  # Counter for found neighbors
sequences_with_less_than_5_neighbors = []  # For sequences with less than 5 valid neighbors

drought_count = 0
control_count = 0  # Count of neighbors in the "control" group

# Process the "drought" sequences
for stress_index in drought_sequences.index:
    adjusted_index = combined_sequences.index.get_loc(stress_index)

    # Search for neighbors
    neighbors = t.get_nns_by_item(adjusted_index, n_neighbors + 1)
    valid_neighbors = []

    for i in neighbors:
        if combined_sequences.iloc[i]['circName'] != combined_sequences.iloc[adjusted_index]['circName']:
            valid_neighbors.append(i)
        if len(valid_neighbors) == n_neighbors:
            break

    # Check if 5 valid neighbors were found
    if len(valid_neighbors) < n_neighbors:
        valid_neighbors_count.append(0)
        drought_without_neighbors += 1
        sequences_with_less_than_5_neighbors.append(combined_sequences.iloc[adjusted_index]['circName'])
    else:
        valid_neighbors_count.append(len(valid_neighbors))

    total_neighbors_found += len(valid_neighbors)  # Total neighbors found
    similarities[stress_index] = valid_neighbors  # Add the found neighbors for the sequence

    # Count neighbors for "drought" and "control"
    for i in valid_neighbors:
        neighbor_condition = combined_sequences.iloc[i]['stress']
        if neighbor_condition == 'drought':
            drought_count += 1
        elif neighbor_condition == 'control':
            control_count += 1

# Display the count of valid neighbors per "drought" sequence
print("Valid neighbors count per 'drought' sequence:")
print(valid_neighbors_count)

# Display the number of sequences with no neighbors
print(f'Number of iterated drought stress sequences without neighbors: {drought_without_neighbors}')
print(f'Total number of neighbors found: {total_neighbors_found}')  # Verifying the total number of neighbors found

# Print the sequences with less than 5 valid neighbors
print("Sequences with less than 5 valid neighbors:")
print(sequences_with_less_than_5_neighbors)

# Creating the final DataFrame
data = []

# Populate the DataFrame with found neighbors
for stress_index, neighbors in similarities.items():
    drought_sequence = combined_sequences.iloc[combined_sequences.index.get_loc(stress_index)]['circName']

    # If the "drought" sequence found less than 5 valid neighbors
    if len(neighbors) == 0:
        data.append({
            'seq_ref_drought': drought_sequence,
            'nearest_neighbor_circrna': '',
            'condition': ''
        })
    else:
        # Add the neighbors to the DataFrame
        for i in range(len(neighbors)):
            neighbor_sequence = combined_sequences.iloc[neighbors[i]]['circName']
            neighbor_condition = combined_sequences.iloc[neighbors[i]]['stress']
            data.append({
                'seq_ref_drought': drought_sequence,
                'nearest_neighbor_circrna': neighbor_sequence,
                'condition': neighbor_condition
            })

# Add sequences with no valid neighbors to the DataFrame
for seq in sequences_with_less_than_5_neighbors:
    data.append({
        'seq_ref_drought': seq,
        'nearest_neighbor_circrna': '',
        'condition': ''
    })

# Create the final DataFrame
df_results = pd.DataFrame(data)

# Similarity analysis
total_neighbors = len(similarities) * n_neighbors
num_drought_sequences = len(drought_sequences)

print(f'Number of iterated drought stress sequences: {num_drought_sequences}')
print(f'Total number of neighbors in the "drought" group: {drought_count}')
print(f'Total number of neighbors in the "control" group: {control_count}')
print(f'Percentage of neighbors in the "drought" group: {drought_count / total_neighbors * 100:.2f}%')
print(f'Percentage of neighbors in the "control" group: {control_count / total_neighbors * 100:.2f}%')

# Verifying the number of rows in the final DataFrame
print(f'Total number of rows in df_results: {len(df_results)}')

df_results

Valid neighbors count per 'drought' sequence:
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

,seq_ref_drought,nearest_neighbor_circrna,condition
0,osa-circ5574-OS05T0563400,osa-circ2039-OS03T0747800,control
1,osa-circ5574-OS05T0563400,osa-circ12645-OS02T0735200,drought
2,osa-circ5574-OS05T0563400,osa-circ12543-OS02T0507400,drought
3,osa-circ5574-OS05T0563400,osa-circ32014-OS03T0107600,control
4,osa-circ5574-OS05T0563400,osa-circ4133-OS01T0558600,control
...,...,...,...
60145,zma-circ37973-Zm00001d048176,zma-circ31638-Zm00001d048176,control
60146,zma-circ37973-Zm00001d048176,zma-circ37908-Zm00001d048176,drought
60147,zma-circ37973-Zm00001d048176,zma-circ37893-Zm00001d048176,drought
60148,zma-circ37973-Zm00001d048176,zma-circ37899-Zm00001d048176,drought


In [24]:
df_results.to_excel('neighbors_results_drought.xlsx', index=False)